# Label Dataset Program

In [3]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np
import sys
from IPython.display import clear_output
from pynput.keyboard import Key

# Organise video into classes

## Sort Files

In [5]:
videoLocation = "D:/Thesis/Video-Segments-AUSLAN"
folderList = os.listdir(videoLocation)
words = {}
wordAmount = {}
videoNum = 0
for videoName in folderList:
    location = videoLocation + '/' + videoName
    videoList = os.listdir(location)
    for videoName in videoList:
        if (videoName.count('_')) == 3:
            videoNum += 1
            try:
                wordAmount[str.split(videoName,'_')[0]] += 1
                words[str.split(videoName,'_')[0]].append(location + '/' + videoName)
            except KeyError:
                wordAmount[str.split(videoName,'_')[0]] = 1
                words[str.split(videoName,'_')[0]] = [location + '/' + videoName]

KeyboardInterrupt: 

## Move files

In [ ]:
f = open("failedToSort.txt", "w")
saveLocation = "D:/Thesis/Video-Auslan-sorted"
for key in words.keys():
    if key == '':
        continue
    wordFolder = saveLocation + '/' + key
    try:
        os.mkdir(wordFolder)
    except OSError as error:
        print(key, 'folder exists')
    for i, videoPath in enumerate(words[key]):
        if os.path.isfile(wordFolder + '/' + words[key][i].split('/')[-1]):
            print(words[key][i].split('/')[-1], 'exists')
        else:
            if os.path.isfile(videoPath):
                try:
                    shutil.copy(videoPath, wordFolder)
                except:
                    f.write(videoPath + '\n')
            else:
                print('cannot copy')
    print(key, 'coppied')
f.close()

#### Check if moved correct

In [6]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
numVid = 0
for folders in os.listdir(saveLocation):
    numVid += len(os.listdir(saveLocation + '/' +folders))

numOriginal = 0
fromLocation = "D:/Thesis/Video-Segments-AUSLAN"
for folders in os.listdir(fromLocation):
    numOriginal += len(os.listdir(fromLocation + '/' + folders))
print('Number of videos Original: ', numOriginal)
print('Number of videos New: ', numVid)
#Loss of videos likley from old file locations had random dummy files 

KeyboardInterrupt: 

### ERRORED FILE

In [4]:
corrupted = ['AJBB1c2b',
'AJPB1c2b',
'AMMA1c2a',
'AWGB1c2a',
'BAOBB1c2a',
'BCPA1c2a',
'BDLA1c2a',
'BFPB1c2a',
'BFSA1c2a',
'BGLB1c2b',
'BRCA1c2a',
'BRVA1c2b',
'BSSA1c2b',
'BCHA2c7a',
'SSNA1c2a']

### Labeler For Path Movements

In [11]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
pathMovement = {}
for word in os.listdir(saveLocation):
    pathMovement[word] = None

In [12]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
finishedWords = {}
for word in os.listdir(saveLocation):
    wordLocation = saveLocation + '/' + word
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        finishedWords[videoName] = False

In [13]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
# on a scale of 1 to 4
usability = {}
for word in os.listdir(saveLocation):
    wordLocation = saveLocation + '/' + word
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        usability[videoName] = None

## Load in paths

In [14]:
f = open("keyBinds.txt", "r")
line = f.readline()
while (line):
    try:
        if line.split("_|_")[1] == 'None\n':
            pathMovement[line.split("_|_")[0]] = None
        else:
            pathMovement[line.split("_|_")[0]] = line.split("_|_")[1]
    except:
        i = 1
    line = f.readline()
f.close()

In [21]:
f = open("useability.txt", "r")
line = f.readline()
while (line):
    try:
        if line.split("_|_")[1] == 'None\n':
            usability[line.split("_|_")[0]] = None
        else:
            usability[line.split("_|_")[0]] = int(line.split("_|_")[1])
    except:
        if line != '\n':
            print(line)
    line = f.readline()
f.close()

In [18]:
f = open("labelBool.txt", "r")
line = f.readline()
while (line):
    try:
        finishedWords[line.split("_|_")[0]] = bool(line.split("_|_")[1])
    except:
        print('error')
    line = f.readline()
f.close()

### Load in KNN predicted usability

In [32]:
knnUsability = pd.read_csv('usabilityKNNPredicted.csv')
knnUsabilityDic = {i:j for i,j  in np.array([np.array(knnUsability).T[1], np.array(knnUsability).T[3]]).T}
usability = knnUsabilityDic

## skip labeled videos

In [36]:
# both usability and pathmovement
for video in usability:
    try:
        if usability[video] != None and pathMovement[video.split("_")[0]] != None:
            finishedWords[video] = True
        else:
            finishedWords[video] = False
    except KeyError:
        print(video.split("_")[0])
finishedWords

AREA-2h
AREA-2h
BAD-2h
HAVE-2h
PUT-2h
PUT-2h
SCARED-2h
SCARED-2h
WALK-1h
YELL2-2h


{'ABOUT1_BFSA1c2a_1630_1990.mp4': True,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': True,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': True,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': True,
 'ABOUT2_AAPB2c7a_310_800.mp4': True,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': True,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': True,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': True,
 'ABOUT2_AMGA1c2a_730_1170.mp4': True,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': True,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': True,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': True,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': True,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': True,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': True,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': True,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': True,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': True,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': True,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': True,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': True,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': True,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': True,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': True,
 'ABOUT2_SBS1A1c2b_7320_7630.mp4': T

In [35]:
# Just pathmovement
for video in usability:
    try:
        if pathMovement[video.split("_")[0]] != None:
            finishedWords[video] = True
        else:
            finishedWords[video] = False
    except KeyError:
        print(video.split("_")[0])
finishedWords

AREA-2h
AREA-2h
BAD-2h
HAVE-2h
PUT-2h
PUT-2h
SCARED-2h
SCARED-2h
WALK-1h
YELL2-2h


{'ABOUT1_BFSA1c2a_1630_1990.mp4': True,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': True,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': True,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': True,
 'ABOUT2_AAPB2c7a_310_800.mp4': True,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': True,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': True,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': True,
 'ABOUT2_AMGA1c2a_730_1170.mp4': True,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': True,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': True,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': True,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': True,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': True,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': True,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': True,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': True,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': True,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': True,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': True,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': True,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': True,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': True,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': True,
 'ABOUT2_SBS1A1c2b_7320_7630.mp4': T

In [39]:
# pathmovement that have usability 3 and over
for video in usability:
    try:
        if usability[video] <= 2:
            finishedWords[video] = True
        elif usability[video] > 2 and pathMovement[video.split("_")[0]] != None:
            finishedWords[video] = True
        else:
            finishedWords[video] = False
    except KeyError:
        print(video.split("_")[0])
finishedWords

AREA-2h
PUT-2h
PUT-2h
WALK-1h


{'ABOUT1_BFSA1c2a_1630_1990.mp4': True,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': True,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': True,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': True,
 'ABOUT2_AAPB2c7a_310_800.mp4': True,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': True,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': True,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': True,
 'ABOUT2_AMGA1c2a_730_1170.mp4': True,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': True,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': True,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': True,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': True,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': True,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': True,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': True,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': True,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': True,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': True,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': True,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': True,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': True,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': True,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': True,
 'ABOUT2_SBS1A1c2b_7320_7630.mp4': T

## Label Program

In [42]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
playbackSpeed = int((1/25)*1000)

 # Loop through every folder
done = False
for word in os.listdir(saveLocation):
    if done:
        break
    nextWord = False
    wordLocation = saveLocation + '/' + word
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        if nextWord or finishedWords[videoName]:
            break
        # Ensure its not corrupted
        isIn = False
        for corruptName in corrupted:
            isIn = isIn or (corruptName in videoName)
        if not isIn:
            # Load video Files
            videoLocation = wordLocation + '/' + videoName
            frames = []
            cap = cv2.VideoCapture(videoLocation)
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()

            # Display video
            if len(frames) > 0:
                frameNum = 0

                # Terminal Title stuff
                extra = [videoName, str(finishedWords[videoName])]
                prevExtra = 0
                if not pathMovement[word] is None:
                    extra.append(pathMovement[word])
                if not usability[videoName] is None:
                    extra.append("Usability: " + str(usability[videoName]))
                else:
                    extra.append("Usability: None" )
                while True:
                    # Display Frame
                    try:
                        cv2.imshow('MediaPipe Pose', frames[frameNum])
                    except:
                        print(videoName, frameNum, len(frames))
                        break

                    # Increment frame
                    frameNum += 1
                    if frameNum >= len(frames):
                        frameNum = 0

                    # Clear terminal
                    if len(extra) != prevExtra:
                        clear_output(wait=True)
                        printStr = ""
                        for wrds in extra:
                            printStr += wrds
                            printStr += " | "
                        print(printStr)
                    prevExtra = len(extra)
                    

                    # Key Press detection
                    
                    k = cv2.waitKey(playbackSpeed) & 0xFF
                    # Label Keys
                    if k == ord('8'):                           # Movement Up
                        extra.append('Up')
                        pathMovement[word] = "Up"
                    elif k == ord('2'):                         # Movement Down
                        extra.append('Down')
                        pathMovement[word] = "Down"
                    elif k == ord('5'):                         # Movement Up and Down
                        extra.append('Up and Down')
                        pathMovement[word] = "Up and Down"
                    elif k == ord('4') or k == ord('6'):        # Movement Sideways
                        extra.append('Sideways')
                        pathMovement[word] = "Sideways"
                    elif k == ord('+'):                         # Movement Side to Side
                        extra.append('Side to Side')
                        pathMovement[word] = "Side to Side"
                    elif k == ord('7') or k == ord('9'):        # Movement Away
                        extra.append('Away')
                        pathMovement[word] = "Away"
                    elif k == ord('1')or k == ord('3'):         # Movement Towards
                        extra.append('Towards')
                        pathMovement[word] = "Towards"
                    elif k == 13:                               # Movement Back and Forth
                        extra.append('Back and Forth')
                        pathMovement[word] = "Back and Forth"
                    elif k == ord('*'):                         # Movement Horizontal Circular
                        extra.append('Horizontal Circular')
                        pathMovement[word] = "Horizontal Circular"
                    elif k == ord('/'):                         # Movement Vertical Circular
                        pathMovement[word] = "Vertical Circular"
                        extra.append('Vertical Circular')
                    elif k == ord('-'):                         # Unknown
                        extra.append('Unknown')
                        pathMovement[word] = "Unknown"
                    elif k == ord('.'):                         # Local Movement
                        extra.append('Local Movement')
                        pathMovement[word] = "Local Movement"


                    # Usability of video
                    elif k == ord('z'):                         # Movement Vertical Circular
                        extra.append('Usability: 1')
                        usability[videoName] = 1
                    elif k == ord('x'):                         # Movement Vertical Circular
                        extra.append('Usability: 2')
                        usability[videoName] = 2
                    elif k == ord('c'):                         # Movement Vertical Circular
                        extra.append('Usability: 3')
                        usability[videoName] = 3
                    elif k == ord(' '):                         # Movement Vertical Circular
                        extra.append('Usability: 4')
                        usability[videoName] = 4

                    # General Keys
                    elif k == ord('q'):     # Next Variation of Sign
                        break
                    elif k == ord('t'):
                        finishedWords[videoName] = True
                    elif k == 27:           # Next Word
                        nextWord = True
                        break
                    elif k == ord(')'):     # Exit fully
                        nextWord = True
                        done = True
                        break
                cv2.destroyAllWindows()
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

HAT(5)-2H_AVBB2c7a_14400_15010.mp4 | False | Usability: 3 | 


In [47]:
cv2.destroyAllWindows()
f = open("keyBinds.txt", "w")
for key in pathMovement:
    if pathMovement[key] is None:
        f.write(key + "_|_" + 'None' + '\n')
    else:
        try:
            f.write(key + "_|_" + pathMovement[key] + '\n')
        except:
            print(pathMovement[key])
f.close()
pathMovement

{'ABOUT1': 'Vertical Circular\n',
 'ABOUT2': 'Vertical Circular\n',
 'ABOUT2(7)': 'Vertical Circular\n',
 'ABOUT2(B)': None,
 'ABOUT2(FALSE-START)': 'Unknown\n',
 'ACCEPT': 'Towards\n',
 'ACCOMMODATION-2H': 'Back and Forth\n',
 'ACCURATE': 'Up and Down\n',
 'ACHIEVE': 'Away\n',
 'ACTIVITY-THERE': 'Sideways\n',
 'ADULT': 'Horizontal Circular\n',
 'AFRAID': 'Back and Forth\n',
 'AFTERNOON': 'Local Movement\n',
 'AFTERNOON3': 'Towards\n',
 'AFTERNOON4': 'Towards\n',
 'AGAIN': 'Down\n',
 'AGAIN(8)': None,
 'AGAIN-2H': 'Local Movement\n',
 'AGAIN-AND-AGAIN': 'Up and Down\n',
 'AGES': 'Horizontal Circular\n',
 'AGREE': 'Sideways\n',
 'AIM': 'Unknown\n',
 'ALIVE': 'Towards\n',
 'ALL': 'Horizontal Circular\n',
 'ALL-2H': 'Away\n',
 'ALL-DAY': 'Up\n',
 'ALL-DAY-2H': 'Horizontal Circular\n',
 'ALL-DAY-LONG': 'Vertical Circular\n',
 'ALLOW': 'Local Movement\n',
 'ALMOST': 'Unknown\n',
 'ALMOST(FLATGO)': None,
 'ALMOST-2H': 'Unknown\n',
 'ALRIGHT': 'Unknown\n',
 'ALRIGHT-2H': 'Unknown\n',
 'ALRIGH

In [45]:
cv2.destroyAllWindows()
f = open("useability.txt", "w")
for video in usability:
    if usability[video] is None:
        f.write(video + "_|_" + 'None' + '\n')
    else:
        f.write(video + "_|_" + str(usability[video]) + '\n')
f.close()
usability

{'ABOUT1_BFSA1c2a_1630_1990.mp4': 0,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': 0,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': 0,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': 3,
 'ABOUT2_AAPB2c7a_310_800.mp4': 2,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': 0,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': 3,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': 1,
 'ABOUT2_AMGA1c2a_730_1170.mp4': 3,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': 1,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': 0,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': 0,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': 0,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': 4,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': 0,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': 4,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': 2,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': 4,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': 4,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': 2,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': 4,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': 4,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': 4,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': 4,
 'ABOUT2_SBS1A1c2b_7320_7630.mp4': 2,
 'ABOUT2_SPKA1c2a_171110_171409.mp4': 1,
 'ABOUT2_STMB1c2a_13650_13800

In [46]:
cv2.destroyAllWindows()
f = open("labelBool.txt", "w")
for video in finishedWords:
    if finishedWords[video] is None:
        f.write(video + "_|_" + 'None' + '\n')
    else:
        f.write(video + "_|_" + str(finishedWords[video]) + '\n')
f.close()
finishedWords

{'ABOUT1_BFSA1c2a_1630_1990.mp4': True,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': True,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': True,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': True,
 'ABOUT2_AAPB2c7a_310_800.mp4': True,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': True,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': True,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': True,
 'ABOUT2_AMGA1c2a_730_1170.mp4': True,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': True,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': True,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': True,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': True,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': True,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': True,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': True,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': True,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': True,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': True,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': True,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': True,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': True,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': True,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': True,
 'ABOUT2_SBS1A1c2b_7320_7630.mp4': T

### test Find corrupted Video - Already found

In [18]:
saveLocation = "D:/Thesis/Video-Auslan-sorted" + '/' + os.listdir("D:/Thesis/Video-Auslan-sorted")[1000]
videoLocation = saveLocation + '/' + os.listdir(saveLocation)[0]

frames = []
cap = cv2.VideoCapture(videoLocation)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()
print(width, height)
test = np.array(frames)
numPixels = test.shape[0] *test.shape[1] * test.shape[2]
sumRGB = [0,0,0]
for frame in test:
    for y in frame:
        for x in y:
            sumRGB[0] += x[0]
            sumRGB[1] += x[1]
            sumRGB[2] += x[2]
sumRGB[0] = sumRGB[0] / numPixels
sumRGB[1] = sumRGB[1] / numPixels
sumRGB[2] = sumRGB[2] / numPixels
sumRGB

480 352


[68.91214922664142, 57.23134706439394, 53.744421164772724]

# Making the label

In [26]:
videoLocation = "D:\Thesis\Video-Auslan-sorted"
saveLocation = "D:\Thesis\ELAR-Data"
folderList = os.listdir(videoLocation)
labels = []
for sign in folderList:
    labels.append(sign)
data = {'Sign': labels, 'Id': [i for i in range(len(labels))]}
labelDf = pd.DataFrame(data)
labelDf.to_csv(saveLocation + '/' + "label.csv", index=False)

In [31]:
labelLocation = 'D:/Thesis/ELAR-Data/label.csv'
labelDf = pd.read_csv(labelLocation)
labelArr = np.array(labelDf)
dict(labelArr)

{'ABOUT1': 0,
 'ABOUT2': 1,
 'ABOUT2(7)': 2,
 'ABOUT2(B)': 3,
 'ABOUT2(FALSE-START)': 4,
 'ACCEPT': 5,
 'ACCOMMODATION-2H': 6,
 'ACCURATE': 7,
 'ACHIEVE': 8,
 'ACTIVITY-THERE': 9,
 'ADULT': 10,
 'AFRAID': 11,
 'AFTERNOON': 12,
 'AFTERNOON3': 13,
 'AFTERNOON4': 14,
 'AGAIN': 15,
 'AGAIN(8)': 16,
 'AGAIN-2H': 17,
 'AGAIN-AND-AGAIN': 18,
 'AGES': 19,
 'AGREE': 20,
 'AIM': 21,
 'ALIVE': 22,
 'ALL': 23,
 'ALL-2H': 24,
 'ALL-DAY': 25,
 'ALL-DAY-2H': 26,
 'ALL-DAY-LONG': 27,
 'ALLOW': 28,
 'ALMOST': 29,
 'ALMOST(FLATGO)': 30,
 'ALMOST-2H': 31,
 'ALRIGHT': 32,
 'ALRIGHT-2H': 33,
 'ALRIGHT2': 34,
 'ALWAYS': 35,
 'ALWAYS1': 36,
 'ALWAYS2': 37,
 'AMERINDIAN': 38,
 'AMERINDIAN(FALSE START)': 39,
 'AMERINDIAN2': 40,
 'AMERINDIAN2(FALSE START)': 41,
 'AND-THEN': 42,
 'AND1': 43,
 'ANGRY': 44,
 'ANGRY-2H': 45,
 'ANGRY1': 46,
 'ANGRY1-2H': 47,
 'ANIMAL': 48,
 'ANNOUNCE': 49,
 'ANNOYED': 50,
 'ANOTHER': 51,
 'ANSWER': 52,
 'APOSTROPHE': 53,
 'APPROACH': 54,
 'APPROACH-CLOSE': 55,
 'AREA': 56,
 'AREA-2H